## Skeleton of Assignment 4:

    test if the distribution of 

    1) trip duration of bikers that ride during the day vs night

    2) age of bikers for trips originating in Manhattan and in Brooklyn

    are different. Use 3 tests: KS, Pearson's, Spearman's. 

    Use the scipy.stats functions scipy.stats.ks_2samp, scipy.stats.pearsonr, scipy.stats.spearmanr. 

    For the KS do the test with the entire dataset and with a subset 200 times smaller

    Choose a single significant threshold for the whole exercise. 
    
    For each test phrase the Null Hypothesis in words.

    Describe the return of the scipy function you use in each case.

    State the result in terms of rejection of the Null.

In [8]:
import pylab as pl
import pandas as pd
import numpy as np
#imports downloader
#from getCitiBikeCSV import getCitiBikeCSV
%pylab inline
import os
import json
import scipy.stats

Populating the interactive namespace from numpy and matplotlib


In [10]:
##Function to download and unpack Citibike data.
##Moves the data to the directory pointed to by $PUIDATA

import os
import sys

#check if the env variable PUIDATA exists
if os.getenv("PUIDATA") is None:
    print ("must set PUIDATA env variable")
    sys.exit()

def getCitiBikeCSV(datestring):
    '''Downloads citibike data and unzips it. If the data is downloaded by not unzippeds it zips it. Moves the data to $PUIDATA
    Arguments:
        date string as yyyymm
    '''
    print ("Downloading", datestring)
    ### First I will heck that it is not already there
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        if os.path.isfile(datestring + "-citibike-tripdata.csv"):
            # if in the current dir just move it
            if os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA")):
                print ("Error moving file!, Please check!")
        #otherwise start looking for the zip file
        else:
            if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip"):
                if not os.path.isfile(datestring + "-citibike-tripdata.zip"):
                    os.system("curl -O https://s3.amazonaws.com/tripdata/" + datestring + "-citibike-tripdata.zip")
                ###  To move it I use the os.system() functions to run bash commands with arguments
                os.system("mv " + datestring + "-citibike-tripdata.zip " + os.getenv("PUIDATA"))
            ### unzip the csv 
            os.system("unzip " + os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip")
            ## NOTE: old csv citibike data had a different name structure. 
            if '2014' in datestring:
                os.system("mv " + datestring[:4] + '-' +  datestring[4:] + 
                          "\ -\ Citi\ Bike\ trip\ data.csv " + datestring + "-citibike-tripdata.csv")
            os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA"))
    ### One final check:
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        print ("WARNING!!! something is wrong: the file is not there!")

    else:
        print("file in place, you can continue")

## Read in the Data

I am reading in data from January 2015 with the function that I created getCitiBikeCSV. You are requested to use 2 months at least. It would be a good idea to use data from a colder and a warmer months, since there are more riders in the warm weather and ridership patterns may change with weather, temperature, etc. You should use data from multiple months, joining multiple datasets (thus addressing some systematic errors as well)

In [12]:
datestring1 = '201603'
datestring2 = '201604'


getCitiBikeCSV(datestring1)
getCitiBikeCSV(datestring2)
df1 = pd.read_csv(os.getenv("PUIDATA") + "/" + datestring1 + '-citibike-tripdata.csv')
df2 = pd.read_csv(os.getenv("PUIDATA") + "/" + datestring2 + '-citibike-tripdata.csv')

df = pd.concat([df1,df2])
df.head()

file in place, you can continue
file in place, you can continue


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1491,3/1/2016 06:52:42,3/1/2016 07:17:33,72,W 52 St & 11 Ave,40.767272,-73.993929,427,Bus Slip & State St,40.701907,-74.013942,23914,Subscriber,1982.0,1
1,1044,3/1/2016 07:05:50,3/1/2016 07:23:15,72,W 52 St & 11 Ave,40.767272,-73.993929,254,W 11 St & 6 Ave,40.735324,-73.998004,23697,Subscriber,1978.0,1
2,714,3/1/2016 07:15:05,3/1/2016 07:26:59,72,W 52 St & 11 Ave,40.767272,-73.993929,493,W 45 St & 6 Ave,40.756800,-73.982912,21447,Subscriber,1960.0,2
3,329,3/1/2016 07:26:04,3/1/2016 07:31:34,72,W 52 St & 11 Ave,40.767272,-73.993929,478,11 Ave & W 41 St,40.760301,-73.998842,22351,Subscriber,1986.0,1
4,1871,3/1/2016 07:31:30,3/1/2016 08:02:41,72,W 52 St & 11 Ave,40.767272,-73.993929,151,Cleveland Pl & Spring St,40.722104,-73.997249,20985,Subscriber,1978.0,1


In [14]:
df['Date'] = pd.to_datetime(df['starttime'])
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Date
0,1491,3/1/2016 06:52:42,3/1/2016 07:17:33,72,W 52 St & 11 Ave,40.767272,-73.993929,427,Bus Slip & State St,40.701907,-74.013942,23914,Subscriber,1982.0,1,2016-03-01 06:52:42
1,1044,3/1/2016 07:05:50,3/1/2016 07:23:15,72,W 52 St & 11 Ave,40.767272,-73.993929,254,W 11 St & 6 Ave,40.735324,-73.998004,23697,Subscriber,1978.0,1,2016-03-01 07:05:50
2,714,3/1/2016 07:15:05,3/1/2016 07:26:59,72,W 52 St & 11 Ave,40.767272,-73.993929,493,W 45 St & 6 Ave,40.756800,-73.982912,21447,Subscriber,1960.0,2,2016-03-01 07:15:05
3,329,3/1/2016 07:26:04,3/1/2016 07:31:34,72,W 52 St & 11 Ave,40.767272,-73.993929,478,11 Ave & W 41 St,40.760301,-73.998842,22351,Subscriber,1986.0,1,2016-03-01 07:26:04
4,1871,3/1/2016 07:31:30,3/1/2016 08:02:41,72,W 52 St & 11 Ave,40.767272,-73.993929,151,Cleveland Pl & Spring St,40.722104,-73.997249,20985,Subscriber,1978.0,1,2016-03-01 07:31:30


In [16]:
df.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender', 'Date'],
      dtype='object')

## Split by Category

H0: There is no statistical difference between the distributions of trip durations for day and night time riders.
$$ \alpha = 0.05 $$

H1: There is a statistical difference between the distributions of trip durations for day and night time riders.

In [18]:
def findtime(hour):
    if hour >= 6 and hour < 18:
        return 0
    else:
        return 1
df["DayOrNight"] = df['Date'].dt.hour.apply(findtime)
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Date,DayOrNight
0,1491,3/1/2016 06:52:42,3/1/2016 07:17:33,72,W 52 St & 11 Ave,40.767272,-73.993929,427,Bus Slip & State St,40.701907,-74.013942,23914,Subscriber,1982.0,1,2016-03-01 06:52:42,0
1,1044,3/1/2016 07:05:50,3/1/2016 07:23:15,72,W 52 St & 11 Ave,40.767272,-73.993929,254,W 11 St & 6 Ave,40.735324,-73.998004,23697,Subscriber,1978.0,1,2016-03-01 07:05:50,0
2,714,3/1/2016 07:15:05,3/1/2016 07:26:59,72,W 52 St & 11 Ave,40.767272,-73.993929,493,W 45 St & 6 Ave,40.756800,-73.982912,21447,Subscriber,1960.0,2,2016-03-01 07:15:05,0
3,329,3/1/2016 07:26:04,3/1/2016 07:31:34,72,W 52 St & 11 Ave,40.767272,-73.993929,478,11 Ave & W 41 St,40.760301,-73.998842,22351,Subscriber,1986.0,1,2016-03-01 07:26:04,0
4,1871,3/1/2016 07:31:30,3/1/2016 08:02:41,72,W 52 St & 11 Ave,40.767272,-73.993929,151,Cleveland Pl & Spring St,40.722104,-73.997249,20985,Subscriber,1978.0,1,2016-03-01 07:31:30,0


In [20]:
df_day = df[df["DayOrNight"] == 0]
df_night = df[df["DayOrNight"] == 1]

In [22]:
#your df should look something like this
# look at these data carefully... you may see someinteresting values!
df.describe()

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/numpy/lib/function_base.py:3403: RuntimeWarning: Invalid value encountered in median
  RuntimeWarning)


,tripduration,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,birth year,gender,DayOrNight
count,1.933070e+06,1.933070e+06,1.933070e+06,1.933070e+06,1.933070e+06,1.933070e+06,1.933070e+06,1.933070e+06,1.709357e+06,1.933070e+06,1.933070e+06
mean,1.048386e+03,8.800252e+02,4.073755e+01,-7.398708e+01,8.653476e+02,4.073641e+01,-7.398564e+01,1.943758e+04,1.977019e+03,1.087634e+00,2.862359e-01
std,1.094378e+04,1.037885e+03,2.214487e-02,1.569833e-02,1.024978e+03,1.911625e-01,3.452256e-01,2.887891e+03,1.176226e+01,5.616894e-01,4.520011e-01
min,6.100000e+01,7.200000e+01,4.067891e+01,-7.401713e+01,7.200000e+01,0.000000e+00,-7.405950e+01,1.452900e+04,1.885000e+03,0.000000e+00,0.000000e+00
25%,3.700000e+02,3.270000e+02,4.072205e+01,-7.399800e+01,3.260000e+02,4.072205e+01,-7.399810e+01,1.695700e+04,NaN,1.000000e+00,0.000000e+00
50%,6.090000e+02,4.480000e+02,4.073902e+01,-7.398889e+01,4.470000e+02,4.073818e+01,-7.398911e+01,1.932400e+04,NaN,1.000000e+00,0.000000e+00
75%,1.037000e+03,5.220000e+02,4.075255e+01,-7.397772e+01,5.210000e+02,4.075206e+01,-7.397791e+01,2.203200e+04,NaN,1.000000e+00,1.000000e+00
max,3.652747e+06,3.249000e+03,4.078721e+01,-7.392989e+01,3.250000e+03,4.078721e+01,0.000000e+00,2.494500e+04,2.000000e+03,2.000000e+00,1.000000e+00


## KS tests to compare 2 samples

http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ks_2samp.html

the KS test in scipy returns the p-value BUT make sure you understand what the NULL is! read the documentation carefully! what is the null hypothesis that you can/cannot reject?

In [24]:
# Entire Dataset
scipy.stats.ks_2samp((df_day["tripduration"]),(df_night["tripduration"]))

Ks_2sampResult(statistic=0.036606704171469162, pvalue=0.0)

FILL IN THE CELL BELOW!

In [26]:
# The null hypothesis can be rejected because the p value is 0. 



The scipy.stats KS test already tells me the significance and the p-value.

The next few cells are here just to show you how you would obtain the same result by hand, but they are not required.

Remember: the Null hypothesis is rejected if

$D_KS(n1,n2) &gt; c(\alpha) \sqrt{\frac{(n1 + n2)}{n1n2}}$

(see class notes) where $c(\alpha$) is the inverse of the KS distribution, and you do not have to know how to get that cause there are tables that list critical values!!

http://www.real-statistics.com/tests-normality-and-symmetry/statistical-tests-normality-symmetry/kolmogorov-smirnov-test/kolmogorov-distribution/

But also this result depends in your choice of binning through, and thustheresultyou get by hand may not be exactly the same as the one the KS returns. Either way: this is how you would calculate the KS statistics by hand.


In [28]:
np.random.seed(5)

In [30]:
size(df_night['tripduration'])

553314

In [32]:
df_daySamp = np.random.choice(df_day["tripduration"],size=553314,replace=False)

# Pulling one in 200: 553314 / 200 = 2766
df_daySamp200 = np.random.choice(df_daySamp,size=2766,replace=False)
df_nightSamp200 = np.random.choice(df_night["tripduration"],size=2766,replace=False)

In [34]:
# Subsample
# The null hypothesis can be rejected because the p value is below 0.05. 

scipy.stats.ks_2samp(df_daySamp200,df_nightSamp200)

Ks_2sampResult(statistic=0.046637744034707107, pvalue=0.0046869602525954346)


## Now retest using a test for correlation.

That will answer a slightly different question though - formulate the NULL appropriately. The tests for correlations (generally) requires the variable to be paired, so that I can tell if x changes does y change similarly. But the datasets are of different size! You will need to reduce them to the same size. You can do that by subsampling of the data: take only 1 ride every of 200, which you can achieve "slicing and broadcasting" the array or using one of the python function (built in python numpy.random.choice() functions for example: Docstring: choice(a, size=None, replace=True, p=None)

Generates a random sample from a given 1-D array

    .. versionadded:: 1.7.0

Parameters ...

But make sure you understand how to use it! there is an option "replace" which you should think about.


H0: There is no significant correlation between trip durations for day and night time riders.
$$ \alpha = 0.05 $$

H1: There is a significant correlation between trip durations for day and night time riders.

## Pearson's test for correlation
 notice that the Pearson's is a pairwise test: the samples need to be a. the same size b. sorted! (how??)

http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html#scipy.stats.pearsonr

In [36]:
df_daySampSort = np.sort(df_daySamp)
df_nightSampSort = np.sort(df_night["tripduration"])

scipy.stats.pearsonr(df_daySampSort, df_nightSampSort)

(0.96747722675962555, 0.0)

In [38]:
# The null hypothesis is rejected because the correlation coefficient of .97 shows almost
# an exact positive linear relationship, while the p-value of 0 shows such a result has no 
# probability of occuring by chance.

## Spearman's test for correlation
http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html#scipy.stats.spearmanr

In [40]:
scipy.stats.spearmanr(df_daySampSort, df_nightSampSort)

SpearmanrResult(correlation=0.99999930407260829, pvalue=0.0)

please comment on the numbers that you get in the light of the scipy manual: what is rerutned? what does it mean??

In [42]:
# The null hypothesis is rejected because the correlation coefficient of .99 shows almost
# an exact positive linear relationship, while the p-value of 0 shows such a result has no 
# probability of occuring by chance.

## Comparing ages between Manhattan and Brooklyn

In [44]:
# Finding citibike trips above southwest manhattan and west of williamsburg
SWlat = 40.704608
wburglong = -73.967590

# manhattan = 1
# brooklyn = 0
def find_borough(lat_long):
    lat,long = lat_long
    if (lat > SWlat) & (long < wburglong):
        return 1
    else:
        return 0

df['borough'] = df[['start station latitude', 'start station longitude']].apply(find_borough,axis=1)

In [46]:
df['age'] = 2017 - df['birth year']
df = df.dropna(subset=['age'],axis=0)
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Date,DayOrNight,borough,age
0,1491,3/1/2016 06:52:42,3/1/2016 07:17:33,72,W 52 St & 11 Ave,40.767272,-73.993929,427,Bus Slip & State St,40.701907,-74.013942,23914,Subscriber,1982.0,1,2016-03-01 06:52:42,0,1,35.0
1,1044,3/1/2016 07:05:50,3/1/2016 07:23:15,72,W 52 St & 11 Ave,40.767272,-73.993929,254,W 11 St & 6 Ave,40.735324,-73.998004,23697,Subscriber,1978.0,1,2016-03-01 07:05:50,0,1,39.0
2,714,3/1/2016 07:15:05,3/1/2016 07:26:59,72,W 52 St & 11 Ave,40.767272,-73.993929,493,W 45 St & 6 Ave,40.756800,-73.982912,21447,Subscriber,1960.0,2,2016-03-01 07:15:05,0,1,57.0
3,329,3/1/2016 07:26:04,3/1/2016 07:31:34,72,W 52 St & 11 Ave,40.767272,-73.993929,478,11 Ave & W 41 St,40.760301,-73.998842,22351,Subscriber,1986.0,1,2016-03-01 07:26:04,0,1,31.0
4,1871,3/1/2016 07:31:30,3/1/2016 08:02:41,72,W 52 St & 11 Ave,40.767272,-73.993929,151,Cleveland Pl & Spring St,40.722104,-73.997249,20985,Subscriber,1978.0,1,2016-03-01 07:31:30,0,1,39.0


## Split by Category

H0: There is no statistical difference between the distributions of ages for Manhattan and Brooklyn riders.
$$ \alpha = 0.05 $$

H1: There is a statistical difference between the distributions of ages for Manhattan and Brooklyn riders.

In [48]:
df_manhattan = df[df["borough"] == 1]
df_brooklyn = df[df["borough"] == 0]

In [50]:
scipy.stats.ks_2samp((df_manhattan["age"]),(df_brooklyn["age"]))

Ks_2sampResult(statistic=0.086052817997637421, pvalue=0.0)

In [ ]:
# The null hypothesis can be rejected because the p value is 0. 

H0: There is no significant correlation between ages of Brooklyn and Manhattan riders.

α=0.05

H1: There is a significant correlation between ages of Brooklyn and Manhattan riders.

In [52]:
size(df_brooklyn['age'])

280652

In [53]:
df_manhattanSamp = np.random.choice(df_manhattan["age"],size=280652,replace=False)

In [54]:
df_manhattanSort = np.sort(df_manhattanSamp)
df_brooklynSort = np.sort(df_brooklyn["age"])

scipy.stats.pearsonr(df_manhattanSort, df_brooklynSort)

(0.99288544736897566, 0.0)

In [55]:
scipy.stats.spearmanr(df_manhattanSort, df_brooklynSort)

SpearmanrResult(correlation=0.99889777968133575, pvalue=0.0)

In [ ]:
# The null hypothesis is rejected because the correlation coefficients of .99 show almost
# an exact positive linear relationship, while the p-values of 0 shows such a result has no 
# probability of occuring by chance.